# CSV file manipulation

this file was just for testing out how to manipulate the data and understand how to extract the information I wanted from it.

In [1]:
import pandas as pd
import pickle

load pickle file

In [2]:
with open("toy_data.pkl", 'rb') as f:
    df = pickle.load(f)

extract data for gene knockout 'glmU' in a new dataframe and output the number of entries in the list of cell masses


In [4]:
glmU = df[df['gene_ko'] == 'glmU']

for _,row in glmU.iterrows():
    print(len(row['cellMass']))

57
55
49
42
66
59


In [9]:
dfNew = df[df['gene_ko']=='glmU']
dfNew = dfNew[['cellMass', 'time', 'generation']]
dfNew['generation'] = dfNew['generation'].apply(lambda x: int(x.split('_')[-1]))
dfNew.sort_values(by=['generation'], inplace=True)


In [11]:
for _,row in dfNew.iterrows():
    c = len(row['cellMass'])
    t = len(row['time'])
    print(f'generation: {row["generation"]}, cellMass: {c}, time: {t}')


generation: 0, cellMass: 42, time: 42
generation: 1, cellMass: 49, time: 49
generation: 2, cellMass: 59, time: 59
generation: 3, cellMass: 57, time: 57
generation: 4, cellMass: 66, time: 66
generation: 5, cellMass: 55, time: 55


using df.explode to open up the nested lists time and cell mass to be able to iterate over this more easily.

then labelling the last time steps of cells to see where cell division occurs.

In [15]:

dfExp = dfNew.explode('cellMass')
dfT = dfNew.explode('time')

dfExp['time'] = dfT['time']

labels = []
for _,row in dfExp.iterrows():
    current_row = row
    try:
        next_index, next_row = next(df.iterrows())
        # Now you can access the next row using `next_row`
        print("Next row:", next_row)
        if row['generation'] == next_row['generation']:
            label = 0
        else:
            label = 1
        labels.append(label)
    except StopIteration:
        # If there are no more rows, handle the StopIteration exception
        print("Reached the end of the DataFrame")
        row['label'] = 1
        break
    
dfExp['label'] = labels

Next row: --TRANS-ACENAPHTHENE-12-DIOL[c]                                                             NaN
1-5-Galactofuran[c]                                                                         NaN
1-6-Galactofuran[c]                                                                         NaN
1-ACYLGLYCEROL-3-P-ACYLTRANSFER-MONOMER[i]    [130, 130, 135, 139, 145, 149, 154, 162, 165, ...
1-AMINO-PROPAN-2-OL[c]                                                                      NaN
                                                                    ...                        
media                                                                                        M9
generation                                                                    generation_000003
gene_ko                                                                                    yidC
cell_id                                                                                  000000
cell_path                     

export new df to csv

In [16]:
dfExp.to_csv("glmU.csv", index=False)